# SparkSQL R Sample - USA Daily Temperatures

In [1]:
Sys.getenv("SPARK_HOME")

[1] "/Users/skalathur/MyApps/spark"

In [2]:
# Set the correct value for SPARK_HOME if not set in your environment
if (nchar(Sys.getenv("SPARK_HOME")) < 1) {
  Sys.setenv(SPARK_HOME = "/Users/skalathur/MyApps/spark")
}

In [3]:
Sys.setenv(SPARK_LOCAL_IP="localhost")

In [4]:
# load the SparkR library (wait until it loads)
library(SparkR, lib.loc = c(file.path(Sys.getenv("SPARK_HOME"), "R", "lib")))


Attaching package: ‘SparkR’

The following objects are masked from ‘package:stats’:

    cov, filter, lag, na.omit, predict, sd, var, window

The following objects are masked from ‘package:base’:

    as.data.frame, colnames, colnames<-, drop, endsWith, intersect,
    rank, rbind, sample, startsWith, subset, summary, transform, union



In [5]:
# Start the Spark Session, wait until it starts
sparkR.session(master = "local[*]", sparkConfig = list(spark.driver.memory = "2g"))

Spark package found in SPARK_HOME: /Users/skalathur/MyApps/spark


Launching java with spark-submit command /Users/skalathur/MyApps/spark/bin/spark-submit   --driver-memory "2g" sparkr-shell /var/folders/s3/hy6_p79n3w1fw802t6ps40qr0000gp/T//Rtmp7jfYHO/backend_portb3e27079c9f 


Java ref type org.apache.spark.sql.SparkSession id 1 

In [6]:
inputFile <- "/temp/datasets/usa_daily_avg_temps.csv"

In [7]:
# Read the csv file as a SparkDataFrame
usaDailyTemps <- read.df(inputFile, source = "csv", 
                         header='true', 
                         inferSchema='true')

usaDailyTemps

SparkDataFrame[state:string, city:string, month:int, day:int, year:int, avgtemp:double]

In [8]:
printSchema(usaDailyTemps)

root
 |-- state: string (nullable = true)
 |-- city: string (nullable = true)
 |-- month: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- year: integer (nullable = true)
 |-- avgtemp: double (nullable = true)


In [9]:
count(usaDailyTemps)

[1] 1174605

In [10]:
head(usaDailyTemps)

state,city,month,day,year,avgtemp
Alabama,Birmingham,1,1,1995,50.7
Alabama,Birmingham,1,1,1996,56.8
Alabama,Birmingham,1,1,1997,60.9
Alabama,Birmingham,1,1,1998,35.6
Alabama,Birmingham,1,1,1999,41.0
Alabama,Birmingham,1,1,2000,59.0


In [11]:
persist(usaDailyTemps, "MEMORY_AND_DISK")

SparkDataFrame[state:string, city:string, month:int, day:int, year:int, avgtemp:double]

In [12]:
# Register the DataFrame as a SQL table.
createOrReplaceTempView(usaDailyTemps, "usaDailyTempsTable")

### Aggregate to find the maximum of avgtemp

In [13]:
query <- "SELECT max(avgtemp) FROM usaDailyTempsTable"
query

[1] "SELECT max(avgtemp) FROM usaDailyTempsTable"

In [14]:
maxAvgTemp <- sql(query)
maxAvgTemp

SparkDataFrame[max(avgtemp):double]

In [15]:
count(maxAvgTemp)

[1] 1

In [16]:
# collect to local data frame
collect(maxAvgTemp)

max(avgtemp)
107.5


In [17]:
# Provide the appropriate column name (MaxValue)

query <- "SELECT max(avgtemp) AS MaxValue FROM usaDailyTempsTable"

maxAvgTemp <- sql(query)
maxAvgTemp

SparkDataFrame[MaxValue:double]

In [18]:
localDf <- collect(maxAvgTemp)
localDf

MaxValue
107.5


In [19]:
# Filter the SparkDataFrame to find the rows with the max value

query <- paste("SELECT * from usaDailyTempsTable WHERE avgtemp = ", 
               localDf[1, 'MaxValue'])
query

[1] "SELECT * from usaDailyTempsTable WHERE avgtemp =  107.5"

In [20]:
maxData <- sql(query)
maxData

SparkDataFrame[state:string, city:string, month:int, day:int, year:int, avgtemp:double]

In [21]:
# collect to local data frame
collect(maxData)

state,city,month,day,year,avgtemp
Arizona,Yuma,7,22,2006,107.5


### Aggregate to find the maximum of avgtemp grouping by Year

In [22]:
query <- "SELECT year, max(avgtemp) AS MaxValue FROM usaDailyTempsTable 
             GROUP BY year"
query

[1] "SELECT year, max(avgtemp) AS MaxValue FROM usaDailyTempsTable \n             GROUP BY year"

In [23]:
maxTempByYear <- sql(query)
maxTempByYear

SparkDataFrame[year:int, MaxValue:double]

In [24]:
count(maxTempByYear)

[1] 21

In [25]:
collect(maxTempByYear)

year,MaxValue
2003,105.8
2007,104.4
2015,105.1
2006,107.5
2013,104.9
1997,100.6
2014,103.8
2004,101.0
1996,104.3
1998,103.0


In [26]:
query <- "SELECT year, max(avgtemp) AS MaxValue FROM usaDailyTempsTable 
               GROUP BY year ORDER BY year"
query


[1] "SELECT year, max(avgtemp) AS MaxValue FROM usaDailyTempsTable \n               GROUP BY year ORDER BY year"

In [27]:
maxTempByYear <- sql(query)
maxTempByYear

SparkDataFrame[year:int, MaxValue:double]

In [28]:
count(maxTempByYear)
collect(maxTempByYear)

[1] 21

year,MaxValue
1995,104.3
1996,104.3
1997,100.6
1998,103.0
1999,100.1
2000,101.6
2001,104.4
2002,102.6
2003,105.8
2004,101.0


### Aggregate to find the maximum of avgtemp grouping by State

In [29]:
query <- "SELECT state, max(avgtemp) AS MaxValue FROM usaDailyTempsTable 
               GROUP BY state ORDER BY state"
query

[1] "SELECT state, max(avgtemp) AS MaxValue FROM usaDailyTempsTable \n               GROUP BY state ORDER BY state"

In [30]:
maxTempByState <- sql(query)
maxTempByState

SparkDataFrame[state:string, MaxValue:double]

In [31]:
count(maxTempByState)

[1] 50

In [32]:
collect(maxTempByState)

state,MaxValue
Alabama,91.5
Alaska,79.5
Arizona,107.5
Arkansas,100.7
California,102.6
Colorado,94.7
Connecticut,89.8
Delaware,89.7
Florida,92.8
Georgia,97.7


### Aggregate to find the number of entries grouping by State

In [ ]:
query <- "SELECT state, count(*) AS count FROM usaDailyTempsTable 
            GROUP BY state ORDER BY count DESC"
query

In [ ]:
stateCounts <- sql(query)
stateCounts

In [ ]:
collect(stateCounts)

### Aggregate to find the number of entries grouping by State and City

In [ ]:
query <- "SELECT state, city, count(*) AS count FROM usaDailyTempsTable 
            GROUP BY state, city ORDER BY state, city"
query

In [ ]:
stateCityCounts <- sql(query)
stateCityCounts

In [ ]:
collect(stateCityCounts)

### Number of cities for each state in the dataset

In [ ]:
# Register the DataFrame as a SQL table.

createOrReplaceTempView(stateCityCounts, "stateCityCountsTable")

In [ ]:
query <- "SELECT state, count(*) AS count FROM stateCityCountsTable 
             GROUP BY state ORDER BY state"
query

In [ ]:
collect(sql(query))

### Create a subset SparkDataFrame for Boston

In [33]:
query <- "SELECT * FROM usaDailyTempsTable WHERE city == 'Boston'"
query

[1] "SELECT * FROM usaDailyTempsTable WHERE city == 'Boston'"

In [34]:
bostonDailyTemps <- sql(query)

bostonDailyTemps

SparkDataFrame[state:string, city:string, month:int, day:int, year:int, avgtemp:double]

In [35]:
count(bostonDailyTemps)

[1] 7624

In [36]:
# Register the DataFrame as a SQL table.

createOrReplaceTempView(bostonDailyTemps, "bostonDailyTempsTable")

#### Boston Average Temperatures By Year

In [37]:
query <- "SELECT year, avg(avgtemp) AS Average FROM bostonDailyTempsTable
           GROUP BY year ORDER BY year"
query

[1] "SELECT year, avg(avgtemp) AS Average FROM bostonDailyTempsTable\n           GROUP BY year ORDER BY year"

In [38]:
bostonAvgTempsByYear <- sql(query)
bostonAvgTempsByYear

SparkDataFrame[year:int, Average:double]

In [39]:
collect(bostonAvgTempsByYear)

year,Average
1995,51.32027
1996,47.71749
1997,50.83863
1998,51.51562
1999,52.33945
2000,50.36148
2001,52.42822
2002,50.41205
2003,49.73014
2004,50.52514


#### Boston Average Temperatures By Year

In [40]:
query <- "SELECT month, avg(avgtemp) AS Average FROM bostonDailyTempsTable 
             GROUP BY month ORDER BY month"
query

[1] "SELECT month, avg(avgtemp) AS Average FROM bostonDailyTempsTable \n             GROUP BY month ORDER BY month"

In [41]:
bostonAvgTempsByMonth <- sql(query)
bostonAvgTempsByMonth

SparkDataFrame[month:int, Average:double]

In [42]:
collect(bostonAvgTempsByMonth)

month,Average
1,29.76667
2,31.47032
3,37.57604
4,47.08413
5,57.57803
6,66.10714
7,73.55038
8,71.68909
9,65.05762
10,54.73456


#### Boston Average Temperatures By Year and Month

In [43]:
query <- "SELECT year, month, avg(avgtemp) AS Average FROM bostonDailyTempsTable 
             GROUP BY year, month ORDER BY year, month"
query

[1] "SELECT year, month, avg(avgtemp) AS Average FROM bostonDailyTempsTable \n             GROUP BY year, month ORDER BY year, month"

In [44]:
bostonAvgTempsByYearAndMonth <- sql(query)
bostonAvgTempsByYearAndMonth

SparkDataFrame[year:int, month:int, Average:double]

In [45]:
collect(bostonAvgTempsByYearAndMonth)

year,month,Average
1995,1,34.51935
1995,2,28.57500
1995,3,38.03871
1995,4,45.42000
1995,5,56.69677
1995,6,68.47667
1995,7,75.57419
1995,8,72.52581
1995,9,62.93667
1995,10,58.07742


#### Boston years in data

In [ ]:
query <- "SELECT distinct(year) FROM bostonDailyTempsTable ORDER BY year"
query

In [ ]:
yearsDF <- collect(sql(query))
yearsDF

In [ ]:
# Stop the SparkSession now
sparkR.session.stop()